In [ ]:
%tensorflow_version 2.x

In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-11-24 05:36:37--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-11-24 05:36:37--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc40d7609a8b0002c6309df3f702.dl.dropboxusercontent.com/zip_download_get/AnJBI8DSE_8daEqBmpqf_U-b9fblZ_EQan7IBc6pHP2WO2iSPh352CI0GpgWFNsFC84RBIg6oZtx-8k-d2O2QHMq359SV-5s4ZwoZWVM4Ebv2g [following]
--2020-11-24 05:36:37--  https://uc40d7609a8b0002c6309df3f702.dl.dropboxusercontent.com/zip_download_get/AnJBI8DSE_8daEqBmpqf_U-b9fblZ_EQan7IBc6pHP2WO2iSPh352CI0GpgWFNsFC84RBIg6oZtx-8k-d2O2

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [5]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

# New Section

In [ ]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [6]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [7]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , How did Quebec nationalists see their province as a nation in the 1960s?                                      1
What is the best method to teach the pronunciation of English words to kids?                                  1
How much mark should we get to pass in NEET exam?                                                             1
Iâm 13 and overweight. Aside from having a healthy diet, what exercises can I do at home to lose weight?    1
How do you get perfect teeth without using braces?                                                            1
                                                                                                             ..
Why are all psychotherapists so bad at their jobs?                                                            1
How many babies are born each day in India?                                                                   1
What are the milk substitutes for baking a cake?                                        

In [8]:
filter.drop([1306122,1306123],0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [9]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [ ]:
filter.drop(['PK'],1,inplace=True)

In [ ]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [11]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [12]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-03-07 11:10:29--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-03-07 11:10:29--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-03-07 11:10:29--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [13]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [14]:
!rm /content/glove.42B.300d.zip

In [15]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [16]:
len(embeding_index['provide'])

300

In [17]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [18]:
len(embeding_index)

1917494

In [19]:
X=filter['question_text']
Y=filter['target']

In [20]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [34]:
class_weight=class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
print(class_weight)

TypeError: ignored

In [39]:
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(class_weight="balanced",classes=np.unique(y_train),y=y_train)
class_weight = dict(zip(np.unique(y_train), class_weights)),
class_weight

({0.0: 0.5329769262001579, 1.0: 8.081058297628807},)

In [40]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: {0.0: 0.5329769262001579, 1.0: 8.081058297628807}}

In [41]:
x_train

1129140     Should I let my 7th grade daughter have a phone?
47881                             What's Justin Bieber like?
1074012    How many merit list releases by BHU in msc phy...
1121122       What is the evolutionary benefit of diversity?
836740     How can I get a degree or diploma in pharmacy ...
                                 ...                        
1143922    Is it enough having one weight training and on...
71602      If the airport runways are made circular, will...
300958     Will Bannon and his merry band of followers be...
460904     What jobs can a major in psychology and a mino...
413266     If the spiritual heart of man is between his t...
Name: question_text, Length: 1044897, dtype: object

In [42]:
from nltk import word_tokenize

In [43]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [44]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

361


In [45]:
np.percentile(sent_lens,99)

44.0

In [46]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [47]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'did': 48,
 'was': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'b

In [48]:
vocab_size

196219

In [49]:
seq_train_matrix.shape

(1044897, 44)

In [50]:
seq_train

[[31, 8, 592, 18, 3714, 1089, 1110, 24, 4, 259],
 [117, 4249, 10282, 42],
 [9, 68, 3833, 694, 11096, 46, 3674, 6, 2028, 449, 1272, 256],
 [2, 3, 1, 4375, 1190, 7, 3669],
 [9, 15, 8, 33, 4, 316, 23, 1776, 6, 4133, 27, 241],
 [9, 15, 8, 118, 4, 3312, 2010],
 [680, 8, 166, 18, 88174, 6, 1, 4687, 7156, 17],
 [9, 11, 14, 423, 4, 3773, 7310, 584],
 [2, 12, 38, 20692, 1029, 417, 470],
 [3, 781, 23, 1896, 12728, 337, 82, 2165, 6, 57],
 [9, 53, 3, 35567, 64363, 200, 13051, 4060],
 [2, 26, 1, 15739, 88175, 116, 6, 670, 1030],
 [2969, 1756, 54, 55, 34, 239, 2, 26, 19, 116],
 [503,
  13,
  4,
  110,
  59,
  4,
  188,
  113,
  782,
  2,
  31,
  8,
  25,
  547,
  2247,
  1067,
  1087,
  4861,
  1073,
  10,
  3320,
  5,
  10283,
  350,
  10,
  25,
  1909,
  13,
  2798],
 [16, 26, 10284, 129, 143, 7491],
 [9, 11, 8, 33, 3518, 136, 820, 13, 310],
 [9, 13964, 12, 14],
 [30, 3, 1, 34, 677, 655, 13, 2158, 288, 710, 3422, 1272, 256],
 [9, 11, 37, 91, 70, 5, 826, 588, 6, 146, 42, 1828, 10, 1828, 5155],
 [12

In [51]:
x_train.shape

(1044897,)

In [52]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [53]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [54]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [55]:
 len(embedding_matrix)

196220

In [56]:
embedding_matrix.shape

(196220, 300)

In [57]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.096087  , -0.29420999,  0.13939001, ..., -0.32539001,
         0.32743999, -0.10444   ],
       [-0.37992999, -0.35486999,  0.18977   , ...,  0.12404   ,
        -0.89981002, -0.19183999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [58]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [59]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding (Embedding)       (None, 44, 300)           58866000  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [60]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [61]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [62]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [63]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2975 - accuracy: 0.8719
Epoch 1: val_loss improved from inf to 0.27773, saving model to /content/weights-01-0.2777.h5
1045/1045 [==============================] - 411s 384ms/step - loss: 0.2975 - accuracy: 0.8719 - val_loss: 0.2777 - val_accuracy: 0.8867
Epoch 2/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2503 - accuracy: 0.8949
Epoch 2: val_loss did not improve from 0.27773
1045/1045 [==============================] - 396s 379ms/step - loss: 0.2503 - accuracy: 0.8949 - val_loss: 0.2803 - val_accuracy: 0.8788
Epoch 3/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2272 - accuracy: 0.9024
Epoch 3: val_loss improved from 0.27773 to 0.22454, saving model to /content/weights-03-0.2245.h5
1045/1045 [==============================] - 406s 389ms/step - loss: 0.2272 - accuracy: 0.9024 - val_loss: 0.2245 - val_accuracy: 0.9093
Epoch 4/15
1045/1045 [==============================] - 

In [68]:
best_model = load_model('/content/weights-10-0.1957.h5')

OSError: ignored

In [ ]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58823400  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [ ]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9616336198632073

In [ ]:
best_model.save('./spam_filter_model_quora_questions.h5')